Although the language class does not change on switching from a single-tape TM to a multi-tape one, the time complexity class does change. Multitape TMs are more efficient for problems involving copying, comparison, reversal, or parallel scanning, where single-tape TMs suffer from repeated long-distance head movement.

Any language decidable in time T(n) on a multitape TM can be decided on a single-tape TM in O(T(n)^2) time.

I am taking 3 example problems and comparing the worst-case time (when the string belongs in the language) for single-tape and multi-tape TMs for some inputs.

### Example 1: Checking for a palindrome
* Strategy for single-tape: For each letter read from the left end, traverse to the right end and check if it is the same letter. Reject if it's not. Accept when the two ends overlap.
* Strategy for 2-tape: Copy the string on tape 1 onto tape 2. Iterate on tape 1 from the left end forwards and on tape 2 from the right end backwards, comparing corresponding letters on each iteration. Reject in case of any unmatch, accept when half the string is traversed.


In [1]:
def single_tape_palindrome(s):
  n = len(s)
  ops = 0

  left = 0
  right = n-1

  while left < right:
    state = s[left]
    ops += (right-left)     # The number of spaces moved by the head from left to right
    ops += 1                # To compare between the left letter and corresponding right letter
    if state != s[right]:
      return -1
    ops += (right-left)     # The number of spaces moved by the head from right to left

    left += 1
    right -= 1

  return ops


def multi_tape_palindrome(s):
  n = len(s)
  ops = 0
  s2 = ""

  for i in range(n):
    letter = s[i]
    ops += 1                # To read the letter from input string on tape 1

    s2 += letter
    ops += 1                # To copy the same letter onto tape 2

  for i in range(n//2):
    letter1 = s[i]
    letter2 = s2[n-i-1]
    ops += 2                # To read letters from both tapes

    if letter1 != letter2:
      return -1
    ops += 1                # To compare the two letters read

  return ops

In [2]:
# Palindromic Case:
for n in [10, 50, 100, 500, 1000]:
    s = "ab" * n + "a"
    print("n = {}".format(2*n+1))
    print("\tMultitape ops:", multi_tape_palindrome(s))
    print("\tSingle tape ops:", single_tape_palindrome(s))
    print()

n = 21
	Multitape ops: 72
	Single tape ops: 230

n = 101
	Multitape ops: 352
	Single tape ops: 5150

n = 201
	Multitape ops: 702
	Single tape ops: 20300

n = 1001
	Multitape ops: 3502
	Single tape ops: 501500

n = 2001
	Multitape ops: 7002
	Single tape ops: 2003000



In [3]:
# Non-palindromic Case:
print(multi_tape_palindrome("ab"))
print(single_tape_palindrome("ab"))

-1
-1


### Example 2: Checking whether a string is of the form *ww*
* Strategy for single-tape: Ensure that the length of the string is even, reject otherwise. For each letter read on the left half of the string, traverse to the corresponding position on the right half and read the letter. Reject if there's a mismatch. Accept after half the string has been read.
* Strategy for multi-tape: Ensure that the length of the string is even, reject otherwise. Copy the first half of the string from tape 1 onto tape 2. Simultaneously iterate through the strings on both tapes checking if corresponding letters are the same. Reject if any mismatch is found, accept after tape 1 been read if tape 2 has also finished at the corresponding letter.

In [4]:
def single_tape_ww(s):
  n = len(s)
  ops = 0

  if n%2 == 1:
    return -1

  half = n // 2
  left = 0

  while left < half:
      state = s[left]
      ops += half                     # To move head from left half position to right half position
      ops += 1                        # To compare the two corresponding letters
      if state != s[left + half]:
          return -1

      ops += half                     # To move the head back to the left half
      left += 1

  return ops

def multi_tape_ww(s):
  n = len(s)
  ops = 0
  s2 = ""
  left = 0
  half = n//2

  if n%2 == 1:
    return -1

  for i in range(half):
    letter = s[i]
    ops += 1                # To read the letter from input string on tape 1

    s2 += letter
    ops += 1                # To copy the same letter onto tape 2

  for i in range(half):
    letter1 = s[i+half]
    letter2 = s2[i]
    ops += 2                # To read letters from both tapes

    if letter1 != letter2:
      return -1
    ops += 1                # To compare the letters read

  return ops

In [5]:
# ww Case:
for n in [10, 50, 100, 500, 1000]:
    s = "ab" * n * 2
    print("n = {}".format(4*n))
    print("\tMultitape ops:", multi_tape_ww(s))
    print("\tSingle tape ops:", single_tape_ww(s))
    print()

n = 40
	Multitape ops: 100
	Single tape ops: 820

n = 200
	Multitape ops: 500
	Single tape ops: 20100

n = 400
	Multitape ops: 1000
	Single tape ops: 80200

n = 2000
	Multitape ops: 5000
	Single tape ops: 2001000

n = 4000
	Multitape ops: 10000
	Single tape ops: 8002000



In [6]:
# Non-ww Case:
print(multi_tape_ww("ab"))
print(single_tape_ww("ab"))

-1
-1


### Example 3: Checking whether an input string has even number of a's
**Modelling assumption:** The TM cannot change states, I'm treating parity as unbounded external memory.
* Strategy for single-tape: Scan the input from left to right to locate an "a". Each time an "a" is found, traverse to a designated region of the tape to flip a parity marker. After updating the marker, traverse back to continue scanning the input. Repeat this process until the entire input has been scanned. Accept if the marker indicates even parity; reject otherwise
* Strategy for 2-tape: Scan the input once from left to right on tape 1. Maintain a parity marker on tape 2 that is flipped each time "a" is encountered. After the input has been fully scanned, accept if the marker indicates even parity; reject otherwise.

In [7]:
def single_tape_even(s):
  n = len(s)
  parity = 0
  ops = 0

  for i in range(n):
    letter = s[i]
    ops += 1                        # To read the input letter

    if letter == "a":
      ops += (n-i)                  # To move to the parity marker
      ops += 1                      # To flip the parity marker
      parity = 1-parity
      ops += (n-i)                  # To move back to the input

  ops += 1                          # To finally check the parity marker
  if parity == 0:
    return ops
  else:
    return -1

def multi_tape_even(s):
  n = len(s)
  ops = 0
  s2 = "0"

  for i in range(n):
    letter = s[i]
    ops += 1                        # To read the input letter

    if letter == "a":
      if s2 == "0":
        s2 = "1"
      else:
        s2 = "0"
    ops += 2                        # To read the parity from tape 2 and write new parity

  ops += 1                          # To finally check the parity marker
  if s2 == "0":
    return ops
  else:
    return -1


In [8]:
# Even-a Case:
for n in [10, 50, 100, 500, 1000]:
    s = "aa" * n
    print("n = {}".format(2*n))
    print("\tMultitape ops:", multi_tape_even(s))
    print("\tSingle tape ops:", single_tape_even(s))
    print()

n = 20
	Multitape ops: 61
	Single tape ops: 461

n = 100
	Multitape ops: 301
	Single tape ops: 10301

n = 200
	Multitape ops: 601
	Single tape ops: 40601

n = 1000
	Multitape ops: 3001
	Single tape ops: 1003001

n = 2000
	Multitape ops: 6001
	Single tape ops: 4006001



In [9]:
# Odd-a Case:
print(multi_tape_even("a"))
print(single_tape_even("a"))

-1
-1
